# Assignment 3 on Natural Language Processing

## Date : 30th Sept, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.



Please submit with outputs. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/NLP

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/NLP


In [2]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

In [3]:
#Load the IMDB dataset. You can load it using pandas as dataframe
df = pd.read_csv('imdb.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def process(sent):
  sent = re.sub(r'<[^>]+>', '', sent)
  sent = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))|(@[^\s]+)|[^\w\s]', '', sent)          
  sent = re.sub(r'\s+', ' ', sent)       
  words = [lemmatizer.lemmatize(token) for token in sent.lower().split(" ")]
  words = [word for word in words if not word in stop_words]                
  return ' '.join(words)

In [6]:
df.loc[:, 'review'] = df.review.apply(lambda x : process(x))
df.head()

,review,sentiment
0,one reviewer ha mentioned watching 1 oz episod...,positive
1,wonderful little production filming technique ...,positive
2,thought wa wonderful way spend time hot summer...,positive
3,basically family little boy jake think zombie ...,negative
4,petter matteis love time money visually stunni...,positive


In [7]:
avg_len = df.review.apply(lambda x : len(x.split(' '))).mean()
classes = {'positive' : df[df.iloc[:, 1]=='positive'].shape[0], 
           'negative' : df[df.iloc[:, 1]=='negative'].shape[0],
           'total' : df.shape[0]}

In [8]:
print(avg_len)
print(classes)

122.30742
{'positive': 25000, 'negative': 25000, 'total': 50000}


# Naive Bayes classifier

In [9]:
# get reviews column from df
reviews = df['review'].tolist()

# get labels column from df
labels = df['sentiment'].tolist()

In [10]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

print(encoder.classes_)

['negative' 'positive']


In [11]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.

train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, test_size=0.2, stratify=labels, random_state=42)

In [12]:
print(len(train_sentences))
print(len(test_sentences))
print(len([label for label in train_labels if label==0]))
print(len([label for label in train_labels if label==1]))
print(len([label for label in test_labels if label==0]))
print(len([label for label in test_labels if label==1]))

40000
10000
20000
20000
5000
5000


Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [13]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words

# max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec = CountVectorizer(max_features = 3000)
X_train = vec.fit_transform(train_sentences)

train_vocab = vec.get_feature_names()
print(train_vocab)
len(train_vocab)

['10', '100', '1010', '11', '12', '13', '14', '15', '18', '1930s', '1950s', '1960s', '1970s', '1980s', '20', '2001', '2005', '2006', '20th', '25', '30', '40', '45', '50', '60', '70', '710', '80', '810', '90', 'abandoned', 'ability', 'able', 'absolute', 'absolutely', 'absurd', 'abuse', 'academy', 'accent', 'accept', 'accepted', 'accident', 'accidentally', 'according', 'account', 'accurate', 'achieve', 'achieved', 'achievement', 'across', 'act', 'acted', 'acting', 'action', 'activity', 'actor', 'actress', 'actual', 'actually', 'ad', 'adam', 'adaptation', 'adapted', 'add', 'added', 'adding', 'addition', 'admit', 'adult', 'advance', 'advantage', 'adventure', 'advice', 'affair', 'affect', 'afraid', 'africa', 'african', 'afternoon', 'age', 'aged', 'agent', 'ago', 'agree', 'ahead', 'aid', 'aint', 'air', 'aired', 'airplane', 'aka', 'al', 'ala', 'alan', 'albeit', 'albert', 'alex', 'alice', 'alien', 'alive', 'allen', 'allow', 'allowed', 'allows', 'almost', 'alone', 'along', 'already', 'alright',

3000

In [14]:
# Use laplace smoothing for words in test set not present in vocab of train set
vec_test = CountVectorizer()
X_test = vec_test.fit_transform(test_sentences)

test_vocab = vec_test.get_feature_names()
print(test_vocab)

['00', '000', '000001', '001', '0010', '002', '007', '00s', '01', '010', '0101', '010707', '010objectionable', '010overall', '0110', '012310', '0126', '02', '039', '048', '05', '0510', '053105', '06', '07', '075', '081006', '09since', '0f', '0ne', '0when', '10', '100', '1000', '10000', '100000', '1000000', '10005000', '1000word', '1001', '100200', '100f', '100hell', '100kin', '100mins', '100percent', '100th', '100thgrade', '100year', '101', '1010', '1010my', '1010overall', '1010s', '1010seek', '1010the', '1011', '1014', '1015', '101503', '101add', '101end', '101not', '101st', '102', '1020', '1025', '102955', '103', '1030', '1035', '104', '105', '105lbs', '105min', '106', '107', '108', '109', '1095', '10acting', '10after', '10bad', '10but', '10dirmichael', '10dirmick', '10dirstefan', '10dirsteve', '10dont', '10fans', '10go', '10great', '10hour', '10i', '10if', '10it', '10just', '10min', '10minutes', '10movie', '10no', '10not', '10note', '10of10', '10one', '10pm', '10th', '10the', '10thl

In [15]:
# Calculating word counts

subset = [X_train[np.equal(train_labels, 0)], X_train[np.equal(train_labels, 1)]]
print(subset[0].shape, subset[1].shape)

vocab_size = 3000
denominator = [np.sum(subset[0]) + vocab_size, np.sum(subset[1]) + vocab_size]    # laplace smoothing + |V|

print(denominator[0], denominator[1])

count = dict(zip(train_vocab, zip(list(np.asarray(np.sum(subset[0], axis=0)).reshape(-1)), 
    list(np.asarray(np.sum(subset[1], axis=0)).reshape(-1)))))

print(count["like"])

for word in test_vocab:
  if (word not in count):
    count[word] = (0, 0)

len(count)

(20000, 3000) (20000, 3000)
1865277 1848741
(17814, 13851)


76168

In [16]:
# likelihood

likelihood = {}

for word in count:
  numerator = [count[word][0] + 1, count[word][1] + 1]

  likelihood[word] = [numerator[0]/denominator[0], numerator[1]/denominator[1]]

likelihood["wonderful"]

[0.00023749823752718764, 0.0011164354552638796]

In [17]:
# Build the model. Don't use the model from sklearn
priors = np.bincount(train_labels) / train_labels.shape[0]
# print(np.log(priors))
print(priors)

def predict(sentence):
  posteriors = np.log(priors)

  for word in sentence.split(" "):
    if (word in likelihood):
      prob = likelihood[word]
      posteriors = posteriors + np.log(prob)
  
  prediction = np.argmax(posteriors)
  return prediction

[0.5 0.5]


In [18]:
# Test the model on test set and report Accuracy
predictions = []
for sentence in test_sentences:
  predictions.append(predict(sentence))

print("Accuracy : {}".format(np.sum(np.equal(predictions, test_labels))/len(test_sentences)))

Accuracy : 0.8424


# *LSTM* based Classifier

Use the above train and test splits.

In [19]:
# Hyperparameters of the model
vocab_size = 3000
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 200
padding_type='post'
trunc_type='post'

In [20]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [21]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 100)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [22]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 111s 99ms/step - loss: 0.3950 - accuracy: 0.8249 - val_loss: 0.3040 - val_accuracy: 0.8765
Epoch 2/5
1125/1125 [==============================] - 110s 98ms/step - loss: 0.2898 - accuracy: 0.8845 - val_loss: 0.3063 - val_accuracy: 0.8740
Epoch 3/5
1125/1125 [==============================] - 111s 99ms/step - loss: 0.2689 - accuracy: 0.8904 - val_loss: 0.3028 - val_accuracy: 0.8700
Epoch 4/5
1125/1125 [==============================] - 110s 98ms/step - loss: 0.2302 - accuracy: 0.9082 - val_loss: 0.3040 - val_accuracy: 0.8723
Epoch 5/5
1125/1125 [==============================] - 110s 98ms/step - loss: 0.2045 - accuracy: 0.9224 - val_loss: 0.3396 - val_accuracy: 0.8780


In [23]:
# Calculate accuracy on Test data
prediction = model.predict(test_padded)

# Get probabilities
print(prediction)

# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = (prediction >= 0.5).astype(int).reshape(-1)
# print(len(pred_labels.shape))
# print(len(test_labels.shape))
# Accuracy : one can use classification_report from sklearn

np.sum(np.equal(pred_labels, test_labels))/pred_labels.shape[0]

[[0.11725983]
 [0.49367344]
 [0.9768711 ]
 ...
 [0.00605314]
 [0.9504723 ]
 [0.5982294 ]]


0.8739

## Get predictions for random examples

In [24]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
predictions = model.predict(padded)

# Get labels based on probability 1 if p>= 0.5 else 0
print((predictions >= 0.5).astype(int))



[[1]
 [0]
 [0]]
